In [1]:
import pandas as pd
from pathlib import Path

In [2]:
routes = pd.read_csv('routes.dat', sep='\t', usecols=['airline','A','B']).drop_duplicates()
airports = pd.read_csv('airport-codes.csv', sep=',')[['iata_code', 'iso_country', 'coordinates']]
airports = pd.concat([airports.drop(columns='coordinates'), airports.coordinates.str.split(', ', expand=True)], axis=1)
airports = airports[~airports.iata_code.isnull()].rename(columns={0:'lon',1:'lat', 'iata_code':'code', 'iso_country':'country'})

def save_df(airlines, name, coloring):
    df = routes[routes.airline.isin(airlines)]
    df = df.merge(airports, how='left', left_on='A', right_on='code').merge(airports, how='left', left_on='B', right_on='code', suffixes=('_A','_B'))
    df = df.drop(columns=['code_A','code_B'])
    df['color'] = df.apply(coloring, axis=1)
    df['AB'] = df.apply(lambda row: f"{row['A']}_{row['B']}", axis=1)

    df = df[(df.country_A=='US') & (df.country_B=='US')]

    df = df.sample(frac=1)
    
    p = Path(name)
    p.mkdir(exist_ok=True)
    df.to_csv(f'{name}/routes.csv', index=False)
    return df

# All

In [3]:
%%time
def coloring(row):
    if row.airline == 'AA': return '#00ffff'
    if row.airline == 'DL': return '#ff0000'
    if row.airline == 'UA': return '#426ae1'

save_df(['AA', 'DL', 'UA'], 'all', coloring)

CPU times: user 89.6 ms, sys: 4.97 ms, total: 94.5 ms
Wall time: 95.6 ms


,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
1347,DL,DTW,OMA,US,-83.35340118408203,42.212398529052734,US,-95.894096,41.3032,#ff0000,DTW_OMA
863,AA,PHL,PHX,US,-75.24109649658203,39.87189865112305,US,-112.01200103759766,33.43429946899414,#00ffff,PHL_PHX
1115,DL,ATL,OAJ,US,-84.428101,33.6367,US,-77.61209869380001,34.8292007446,#ff0000,ATL_OAJ
271,AA,DFW,CHS,US,-97.038002,32.896801,US,-80.04049683,32.89860153,#00ffff,DFW_CHS
147,AA,CLT,SRQ,US,-80.94309997558594,35.2140007019043,US,-82.55439758300781,27.39539909362793,#00ffff,CLT_SRQ
...,...,...,...,...,...,...,...,...,...,...,...
890,AA,PHL,TYS,US,-75.24109649658203,39.87189865112305,US,-83.9940033,35.81100082,#00ffff,PHL_TYS
1408,DL,JFK,CMH,US,-73.7789,40.639801,US,-82.891899,39.998001,#ff0000,JFK_CMH
2323,UA,IAH,RDU,US,-95.34140014648438,29.984399795532227,US,-78.7874984741211,35.877601623535156,#426ae1,IAH_RDU
2127,UA,IAD,BNA,US,-77.45580292,38.94449997,US,-86.6781997680664,36.1245002746582,#426ae1,IAD_BNA


# American

In [4]:
%%time
def coloring(row):
    hubs = ['CLT','ORD','DFW','LAX','MIA','LGA','PHL', 'PHX', 'DCA']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#6cff7d', '#426ae1', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['AA'], 'american', coloring)

CPU times: user 73.2 ms, sys: 3.19 ms, total: 76.4 ms
Wall time: 75.8 ms


,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
783,AA,ORD,SYR,US,-87.9048,41.9786,US,-76.1063003540039,43.11119842529297,#ffff00,ORD_SYR
334,AA,DFW,JAX,US,-97.038002,32.896801,US,-81.68789672851562,30.49410057067871,#ffa500,DFW_JAX
332,AA,DFW,JAC,US,-97.038002,32.896801,US,-110.73799896240234,43.6072998046875,#ffa500,DFW_JAC
862,AA,PHL,PHF,US,-75.24109649658203,39.87189865112305,US,-76.49299622,37.13190079,#fd68b3,PHL_PHF
907,AA,PHX,DSM,US,-112.01200103759766,33.43429946899414,US,-93.66310119628906,41.534000396728516,#426ae1,PHX_DSM
...,...,...,...,...,...,...,...,...,...,...,...
176,AA,DCA,CLE,US,-77.037697,38.8521,US,-81.8498001099,41.4117012024,#6cff7d,DCA_CLE
808,AA,PHL,BTV,US,-75.24109649658203,39.87189865112305,US,-73.15329742429999,44.471900939899996,#fd68b3,PHL_BTV
556,AA,LGA,RDU,US,-73.872597,40.777199,US,-78.7874984741211,35.877601623535156,#426ae1,LGA_RDU
735,AA,ORD,LEX,US,-87.9048,41.9786,US,-84.60590362548828,38.0364990234375,#ffff00,ORD_LEX


# Delta

In [5]:

def coloring(row):
    hubs = ['ATL','BOS','DTW','JFK','LAX','LGA','MSP','SEA','SLC']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#007f00', '#ffddad', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['DL'], 'delta', coloring)

,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
566,DL,LGA,LEX,US,-73.872597,40.777199,US,-84.60590362548828,38.0364990234375,#ffddad,LGA_LEX
536,DL,LAX,SJC,US,-118.407997,33.942501,US,-121.929001,37.362598,#007f00,LAX_SJC
677,DL,MSP,RHI,US,-93.221802,44.882,US,-89.46749877929688,45.63119888305664,#fd68b3,MSP_RHI
765,DL,SLC,BNA,US,-111.97799682617188,40.78839874267578,US,-86.6781997680664,36.1245002746582,#6cff7d,SLC_BNA
433,DL,JFK,BUF,US,-73.7789,40.639801,US,-78.73220062,42.94049835,#00ffff,JFK_BUF
...,...,...,...,...,...,...,...,...,...,...,...
562,DL,LGA,GSP,US,-73.872597,40.777199,US,-82.2189025879,34.8956985474,#ffddad,LGA_GSP
416,DL,HNL,MSP,US,-157.924228,21.32062,US,-93.221802,44.882,#fd68b3,HNL_MSP
96,DL,ATL,IAD,US,-84.428101,33.6367,US,-77.45580292,38.94449997,#ff0000,ATL_IAD
259,DL,CLE,JFK,US,-81.8498001099,41.4117012024,US,-73.7789,40.639801,#00ffff,CLE_JFK


# United

In [6]:

def coloring(row):
    hubs = ['ORD','DEN','IAH','LAX','EWR','SFO','IAD']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#6cff7d', '#426ae1', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['UA'], 'united', coloring)

,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
694,UA,ORD,MSY,US,-87.9048,41.9786,US,-90.25800323486328,29.99340057373047,#ff0000,ORD_MSY
460,UA,IAH,JAX,US,-95.34140014648438,29.984399795532227,US,-81.68789672851562,30.49410057067871,#ffa500,IAH_JAX
668,UA,ORD,HSV,US,-87.9048,41.9786,US,-86.775100708008,34.637199401855,#ff0000,ORD_HSV
609,UA,ORD,AVL,US,-87.9048,41.9786,US,-82.541801,35.436199,#ff0000,ORD_AVL
527,UA,IAH,SHV,US,-95.34140014648438,29.984399795532227,US,-93.8256,32.446602,#ffa500,IAH_SHV
...,...,...,...,...,...,...,...,...,...,...,...
300,UA,HNL,DEN,US,-157.924228,21.32062,US,-104.672996521,39.861698150635,#ffff00,HNL_DEN
616,UA,ORD,BOI,US,-87.9048,41.9786,US,-116.223,43.5644,#ff0000,ORD_BOI
744,UA,ORD,TUL,US,-87.9048,41.9786,US,-95.88809967041016,36.19839859008789,#ff0000,ORD_TUL
830,UA,SFO,STL,US,-122.375,37.61899948120117,US,-90.370003,38.748697,#426ae1,SFO_STL
